### Drosophila melanogaster genes 

To execute bash code we created a snippets through nbextensions containing the following paths in order to avoid copy and paste cell contains. %%bash magic do not recognize previous variables. Adding mkdir command to create necesary paths

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
BASIC='/home/jmurga/mkt/201902/rawData/annotations/basicAnnotation'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

mkdir -p ${DATA}
mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}

Defining global path to python. In this case these variables will be stored after one execution

In [7]:
SRC='/home/jmurga/mkt/201902/scripts/src'
DATA='/home/jmurga/mkt/201902/rawData'

Required libraries

In [8]:
import os
import re
import sys
import numpy as np
import pandas as pd
from numpy import array 
from pyfaidx import Fasta

#### Download Flybase annotation

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'

cd ${DATA}
# Flybase annotation
wget ftp://ftp.flybase.net/genomes/Drosophila_melanogaster/dmel_r5.57_FB2014_03/gff/dmel-all-no-analysis-r5.57.gff.gz
gunzip dmel-all-no-analysis-r5.57.gff.gz

#### Parsing and cleaning flybase annotation to execute all operations by chr
Next operation will be faster due to grep on smaller files. Space is not a problem in our server right now. Each folder contain an specific file foreach chromosome. 

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
BASIC='/home/jmurga/mkt/201902/rawData/annotations/basicAnnotation'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}

# Extract sequences f
# Extract annotations from gff file
sed -e '/^>/,$d' ${DATA}/dmel-all-no-analysis-r5.57.gff | sed -e '/^2LHet/d' -e  '/2RHet/d' -e '/3LHet/d' -e '/3RHet/d' -e  '/^4\t/d' -e '/dmel_mitochondrion_genome/d' -e '/^U\t/d' -e '/^Uextra\t/d' -e  '/^XHet\t/d' -e '/^YHet\t/d' -e'/\tCG/d' > ${DATA}/dmelFiltered.gff
# Extract genes information
grep -P "\tgene\t" ${DATA}/dmelFiltered.gff  > ${DATA}/dmelFilteredGenes.gff
# Extract CDS information. Only protein coding genes
grep -P "\tCDS\t" ${DATA}/dmelFiltered.gff  > ${DATA}/dmelFilteredCds.gff
# Coding gene list. gene_id always on column 9, 3th field
cut -f1,9 ${DATA}/dmelFilteredCds.gff | tr ';' '\t' | cut -f1,2 | sort -u | sort -k1,1 > ${DATA}/codingGeneList.txt


CHR=( 2L 2R 3L 3R X )
# Parse gff file by chr
for chrNumber in "${CHR[@]}"
do
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredGenes.gff | sort -k4,4n > ${BASIC}/gencode.v27lift37.basic.annotation.chr${chrNumber}.gff3 
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredCds.gff | sort -k1,1 -k4,4n > ${CDS}/dmelFilteredCdsChr${chrNumber}.gff3 
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredGenes.gff | sort -k1,1 -k4,4n > ${GENES}/dmelFilteredGenesChr${chrNumber}.gff3 
done

rm ${DATA}/dmel-all-no-analysis-r5.57.gff

#### Basic cleaned gene file
This file will include information about chromosomes, start coordinates, end coordinates strand, gene id and gene name. It will be and perform calculations on gene coordinates. Kind of gff file easier to work with

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
GENES='/home/jmurga/mkt/201902/rawData/annotations/genes'
touch ${DATA}/flybaseGenesCleaned.tab 
printf "chr\tstartGene\tendGene\tstrand\tid\tname\n" > ${DATA}/flybaseGenesCleaned.tab

time while read LINE;
do 
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f2 | sed 's/-cds//g')
    # echo $GENE

    fgrep `echo "${GENE};"`  ${GENES}/dmelFilteredGenesChr${CHR}.gff3 | fgrep ${CHR} |  cut -f1,4,5,7,9 | tr ';' '\t' | cut -f1,2,3,4,5,6 
    
done < ${DATA}/codingGeneList.txt | tr ' ' '\t' | sed 's/ID=//g' | sed 's/Name=//g' | sort -k1,1 -k2,2n >> ${DATA}/flybaseGenesCleaned.tab

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,sep='\t')
dfGenes.head()

File to merge features annotated with gene name instead of gene id

In [ ]:
idName = dfGenes[['id','name']]
idName.to_csv(DATA + '/annotations/idName.tab',sep='\t',index=False,header=True)

#### Extracting CDS coordinates from largest transcript by gene
From the largest transcript we recover id, size, and CDS coordinates. Moreover we save number of transcripts too.  
Coordinates will be used to recover CDS sequences and to calcute Derived Allele Frequencies from 0fold and 4fold degenerate sequences

##### Raw cds coordinates

In [ ]:
%%bash
DATA='/home/jmurga/mkt/201902/rawData/annotations'
CDS='/home/jmurga/mkt/201902/rawData/annotations/cds'
TEMPORAL='/home/jmurga/mkt/201902/rawData/annotations/tmp'

touch ${DATA}/cdsCoordinates.tab
printf "name\tchr\tnumberOfTranscript\ttranscript\ttranscriptSize\tcoordinates\n" > ${DATA}/cdsCoordinates.tab

count=0
time tail -n+2 ${DATA}/flybaseGenesCleaned.tab | while read LINE;
do 
    echo '*************'    
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f6)
    
    printf ${count}
    # fgrep ${GENE} ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | cut -f9 | cut -d';' -f2 | cut -d'=' -f2 | sort -u > ${TEMPORAL}/transcriptTmp.tab
    
    fgrep `echo "Name=${GENE}-cds;"` ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | grep -P "${CHR}\t" | cut -f9 | tr ';' '\n' | fgrep Parent | sort -u | tr ',' '\n' | cut -d'=' -f2 | sort -u > ${TEMPORAL}/transcriptTmp.tab

    largestTranscript=$(while read transcript; do fgrep ${transcript} ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | awk '{print $5-$4}' | awk -v transcript="$transcript"  '{sum+=$1} END{print transcript"\t"sum}' ;done < ${TEMPORAL}/transcriptTmp.tab | sort -nrk2,2 | head -1 )
    
    paste <(echo $GENE) <(echo $CHR) <(wc -l ${TEMPORAL}/transcriptTmp.tab | cut -d' ' -f1) <(echo $largestTranscript | cut -d' ' -f1) <(echo $largestTranscript | cut -d' ' -f2) <(fgrep `echo ${largestTranscript} | cut -d' ' -f1` ${CDS}/dmelFilteredCdsChr${CHR}.gff3  | cut -f4,5 | sort -k1,1n | awk '{printf $1","$2","}' | sed 's/.$//') >> ${DATA}/cdsCoordinates.tab
    
    (( count++ ))

done 


##### N calls distributions

###### N calls distribution by lines

In [ ]:
nCallsX = []

for index, row in cds[cds['chr']=='X'].iterrows():
    print(index)
    # Convert CDS list into numeric array
    coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
    coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    
    
    # Open ref and outgroup
    ref = Fasta(DATA + '/fastas/ref/Chr' + row['chr'] +'.fasta')  
    ## Extract ref and outgroup seq
    refSeq = ref.get_spliced_seq(row['chr'], coordinates).seq.upper()    

    if(('N' not in refSeq) and (len(refSeq)/3).is_integer()):

        # Open population multifasta
        popFasta = Fasta(DATA + '/fastas/alignments/RAL/' + 'RAL_' + 'Chr' + row['chr'] +'.seq')

        #Extract samples
        samples = list(popFasta.keys())

        deletedIndex = 0
        if(row['strand'] == '-'):            

            for i in range(0,len(samples),1):
                tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                tmp = reverseComplement(tmp)
                if(tmp == ('N' * len(refSeq))):
                    deletedIndex+=1

        else:

            for i in range(0,len(samples),1):
                tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                if(tmp == 'N' * len(tmp)):
                    # Save lines index with only N
                    deletedIndex+=1
        nCallsX.append(deletedIndex)
    else:
        continue

In [ ]:
nCalls = pd.DataFrame(nCalls).value_counts().reset_index()


###### N call distrubitution by position

In [ ]:
nCallsByPositions = []

for index, row in cds[(cds['chr']=='2L')].iterrows():
    
# for index, row in cds.iterrows():
    print(row['id'])
    # Convert CDS list into numeric array
    coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
    coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    
    # Open ref and outgroup
    ref = Fasta(DATA + '/fastas/ref/Chr' + row['chr'] +'.fasta')  
    ## Extract ref and outgroup seq
    refSeq = ref.get_spliced_seq(row['chr'], coordinates).seq.upper()    

    if(('N' not in refSeq) and (len(refSeq)/3).is_integer()):
        # Open population multifasta
        popFasta = Fasta(DATA + '/fastas/alignments/RAL/' + 'RAL_' + 'Chr' + row['chr'] +'.seq')

        #Extract samples
        samples = list(popFasta.keys())

        matrixDna = np.empty([len(samples)+1,len(refSeq)],dtype='str')
            
        if(row['strand'] == '-'):            
            refSeq = reverseComplement(refSeq)
            matrixDna[0] = list(refSeq)


            for i in range(0,len(samples),1):
                tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                tmp = reverseComplement(tmp)
                matrixDna[i+1] = list(tmp)


        else:
            matrixDna[0] = list(refSeq)

            for i in range(0,len(samples),1):
                tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
                matrixDna[i+1] = list(tmp)

        # Count occurences
        df = pd.DataFrame(matrixDna)
        for i in df.columns:
            nCallsByPositions.append(df[i].value_counts()['N'])

    else:
        continue

##### Recoding fasta sequences

Opening cdsCoordinates file to extract fasta sequence using pyfaidx

In [9]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,usecols=['id','name','chr','strand'],sep='\t')

cds = pd.read_csv(DATA + '/annotations/cdsCoordinates.tab',header=0,usecols=['name','chr','coordinates'],sep='\t')

cds = pd.merge(cds, dfGenes,  how='inner', left_on=['chr','name'], right_on = ['chr','name'])

In [10]:
cds.shape

(13753, 5)

In [11]:
import sys
sys.path.insert(0, SRC)
from reverseComplement import reverseComplement
from degenerate import degenerate
from dafResampling import dafResampling

In [ ]:
# Rewrite file each execution
columns = ['id','rawDerivedAllele','type']
dmelDaf = pd.DataFrame(columns=columns)
columns = ['id','div','type']
dmelDiv = pd.DataFrame(columns=columns)

import time
start_time = time.time()
for index, row in cds[(cds['chr']=='2L')].head(26).iterrows():
# for index, row in cds.iterrows():
    # Convert CDS list into numeric array
    print(index,row['id'])
    coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
    coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
    
    # Open ref and outgroup
    ref = Fasta(DATA + '/fastas/ref/Chr' + row['chr'] +'.fasta')
    outgroup = Fasta(DATA + '/fastas/outgroup/dsim/Simulans_Chr' + row['chr'] +'.seq')
    
    ## Extract ref and outgroup seq
    refSeq = ref.get_spliced_seq(row['chr'], coordinates).seq.upper()
    outgroupSeq = outgroup.get_spliced_seq('dsim' + row['chr'], coordinates).seq.upper()
   
    # Open population multifasta
    popFasta = Fasta(DATA + '/fastas/alignments/RAL/' + 'RAL_' + 'Chr' + row['chr'] +'.seq')
    
    #Extract samples
    samples = list(popFasta.keys())

    # Open variables and file to write
#     multifasta = open(DATA + '/fastas/genes/RAL/' + row['id'] +'.fa', 'w')
    ## Empty matrix to append ref, pop
    matrix0f=np.empty([len(samples)+1,len(refSeq)],dtype='str')
    matrix4f=np.empty([len(samples)+1,len(refSeq)],dtype='str')
    
    # Outgroup sequences to pd.DataFrame
    outgroup = np.empty([1,len(refSeq)],dtype='str')
    
    if(row['strand'] == '-'):
    #     if((len(refSeq)/3).is_integer()):
        refSeq = reverseComplement(refSeq)
        refSeq0f,refSeq4f = degenerate(refSeq)
        outgroupSeq = reverseComplement(outgroupSeq)

        matrix4f[0] = list(refSeq4f)        
        matrix0f[0] = list(refSeq0f)
        
        # Iter by row matrix to input sequences
        deleteIndex = []
        for i in range(0,len(samples),1):
            tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
            tmp = reverseComplement(tmp)
            if(tmp == ('N' * len(tmp))):
                deleteIndex.append(i+1)
            else:
                matrix4f[i+1] = list(tmp)
                matrix0f[i+1] = list(tmp)
                
        matrix4f = np.delete(matrix4f,deleteIndex,0)
        matrix0f = np.delete(matrix0f,deleteIndex,0)
        
    else:

        refSeq0f,refSeq4f = degenerate(refSeq)
       
        matrix4f[0] = list(refSeq4f)        
        matrix0f[0] = list(refSeq0f)
        
        
        # Iter by row matrix to input sequences
        deleteIndex = []
        for i in range(0,len(samples),1):
            tmp = popFasta.get_spliced_seq(samples[i], coordinates).seq.upper()
            if(tmp == ('N' * len(tmp))):
                deleteIndex.append(i+1)
            else:
                matrix4f[i+1] = list(tmp)
                matrix0f[i+1] = list(tmp)
                
        matrix4f = np.delete(matrix4f,deleteIndex,0)
        matrix0f = np.delete(matrix0f,deleteIndex,0)
        # Iter by matrix column to degenerate all sequences based on reference sequence
    
    # OutputSeq to pd.DataFrame
    outgroup[0] = list(outgroupSeq)
    outgroupDf = pd.DataFrame(outgroup)
    # 0-4fold positions to pd.DataFrame
    df4f = pd.DataFrame(matrix4f)
    df4f = df4f.loc[:,df4f.iloc[0]!='N']
    df0f = pd.DataFrame(matrix0f)
    df0f = df0f.loc[:,df0f.iloc[0]!='N']
    
    daf4f,div4f = dafWithResampling(id=row['id'],data=df4f,outgroup=outgroupDf,resamplingValue=160,type='4fold')
#     daf0f,div0f = dafWithResampling(id=row['id'],data=df0f,outgroup=outgroupDf,resamplingValue=160,type='0fold')
    
    dmelDaf = pd.concat([dmelDaf,daf4f])
    dmelDiv = pd.concat([dmelDiv,div4f])
dmelDaf.to_csv('/home/jmurga/testDaf.csv',sep='\t',header=True,index=False)
print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
    if(df4f.shape[0] < 160):
#         print(df4f.shape)
        continue
    else:
        print(len(refSeq))
        # Delete reference sequence
        ref = df4f.iloc[[0]]
        outgroupToDf = outgroupToDf.loc[:,df4f.columns].rename(index={0:206})
        
        for j in df4f.columns:
            div4f = 0
            if(df4f[[j]][df4f[[j]] != 'N'].dropna().shape[0] < 160):
                continue
            else:
                #Sampling
                tmp = df4f[[j]][df4f[[j]]!='N'].dropna().sample(160,replace=False)
                # Merging outgroup
                tmp = pd.concat([tmp,outgroupToDf[[j]]])
                pos = pd.concat([ref[[j]],tmp]).reset_index(drop=True)
                
                if(pos.loc[len(pos)-1,j]=='N' or pos.loc[len(pos)-1,j]=='-'):
                    continue
                elif((pos.loc[len(pos)-1,j] != pos.loc[0,j]) & (len(pos.loc[1:len(pos)-2,j].unique())==1)): 
                    div4f = 1
                    
                else:

                    AA = pos.loc[len(pos)-1,j]
                    AN = 160
                    AC = pos.loc[1:len(pos)-2,j].value_counts()

                    if(AA not in AC.index):
                        continue
                    else:
                        AC = AC[AC!=AC[AA]]
                        if(len(AC) == 0):
                            af=0
                        else:
                            af=AC[0]/AN
            tmp = pd.Series({'id':row['id'],'rawDerivedAllele':af,'div':div4f,'type':'4fold'})
            dmelDaf = pd.concat([dmelDaf,tmp])